# CNN for many subjects

In [1]:
%pylab
%matplotlib inline

import glob
import os
import mne
CORPORA_PATH = "~/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/"

file_path = os.path.expanduser(CORPORA_PATH)
files = glob.glob(os.path.join(file_path, "*.set"))

def normalize_subject(X):
    mean = X.mean(axis=(0, 2)).reshape(-1, 1)
    std = X.std(axis=(0, 2)).reshape(-1, 1)
    return (X - mean) / std

def load_data(filename, normalize=True):
    data_mne = mne.io.read_raw_eeglab(filename, preload=True, event_id={"0": 1, "1": 2})
    data_mne.filter(0, 20)
    events = mne.find_events(data_mne)
    epochs = mne.Epochs(
        data_mne, events,
        baseline=(None, 0), tmin=-0.1, tmax=0.7)

    epochs.load_data()
    
    ch_names = epochs.ch_names
    
    X = epochs.get_data()[:, :-1]
    y = (events[:, 2] == 2).astype('float')

    if normalize: 
        X = normalize_subject(X)
    
    return X, y 


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


Targets appear as 2 in the third column


We remove last channel as well

# CNN with more data

In [2]:
from IPython.core.debugger import set_trace as bp

filenames = files[:10]


X = None
y = None
print(filenames)
for filename in filenames:
    X_subject, y_subject = load_data(filename)
    
    if X is None:
        X, y = X_subject, y_subject
    else:
        print(X.shape, X_subject.shape)
        X = np.vstack((X, X_subject))
        print(y.shape, y_subject.shape)
        y = np.vstack((y.reshape(-1,1), y_subject.reshape(-1,1)))

['/home/jmperez/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/PruebasMuseo_29426001.set', '/home/jmperez/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/PruebasMuseo_12521001.set', '/home/jmperez/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/PruebasMuseo_14023001.set', '/home/jmperez/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/PruebasMuseo_18531001.set', '/home/jmperez/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/PruebasMuseo_16893001.set', '/home/jmperez/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/PruebasMuseo_945001.set', '/home/jmperez/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/PruebasMuseo_23794001.set', '/home/jmperez/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/PruebasMuseo_1414001.set', '/home/jmperez/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/PruebasMuseo_5568001.set', '/home/jmperez/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/PruebasMuseo_27946001.set']
Reading /home/jmperez/projects/co

In [3]:
X.shape

(23580, 14, 104)

In [4]:
X_t = X[:, :, :, np.newaxis]

X_t.shape

(23580, 14, 104, 1)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X_t, y, test_size=0.1, stratify=y)

X_train.shape, y_train.shape

((21222, 14, 104, 1), (21222, 1))

In [6]:
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Flatten, Dense, Dropout

model = Sequential()

n_kernels = 10
model.add(Conv2D(n_kernels, (14, 1), padding='same', 
                activation='relu', input_shape=(14, 104, 1)))
model.add(Conv2D(5*n_kernels, (1, 13), padding='same',
                activation='relu'))
model.add(Flatten())
model.add(Dropout(0.55))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='adam', 
              metrics=['accuracy']) # reporting the accuracy


Using TensorFlow backend.
/home/jmperez/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [7]:
model.fit(X_train, y_train, epochs=10, batch_size=64, class_weight={0:1, 1:6})

Epoch 1/10
21222/21222 [==============================] - 88s 4ms/step - loss: 1.2710 - acc: 0.3392
Epoch 2/10
21222/21222 [==============================] - 91s 4ms/step - loss: 1.2104 - acc: 0.5309
Epoch 3/10
21222/21222 [==============================] - 90s 4ms/step - loss: 1.1759 - acc: 0.5698
Epoch 4/10
21222/21222 [==============================] - 85s 4ms/step - loss: 1.1210 - acc: 0.6296
Epoch 5/10
21222/21222 [==============================] - 85s 4ms/step - loss: 1.0518 - acc: 0.6784
Epoch 6/10
21222/21222 [==============================] - 85s 4ms/step - loss: 0.9650 - acc: 0.7236
Epoch 7/10
21222/21222 [==============================] - 89s 4ms/step - loss: 0.8866 - acc: 0.7522
Epoch 8/10
21222/21222 [==============================] - 85s 4ms/step - loss: 0.7534 - acc: 0.8036
Epoch 9/10
21222/21222 [==============================] - 85s 4ms/step - loss: 0.6599 - acc: 0.8340
Epoch 10/10
21222/21222 [==============================] - 85s 4ms/step - loss: 0.5494 - acc: 0.8683

In [8]:
y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)
#np.hstack((y_prob, y_pred, y_test.reshape(-1, 1)))

from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.7039864291772688
Precision  = 0.22621184919210055
Recall     = 0.32061068702290074
ROC AUC    = 0.592593024234537

